In [1]:
import sqlite3
from datetime import date
hot_date = date.today()
DATABASE = "chessdelite.db"
db_connection = sqlite3.connect(DATABASE) 
chess_db = db_connection.cursor()

## Database commands for ad hoc use

In [ ]:
chess_db.close()
db_connection.close()

In [ ]:
db_connection.rollback()

In [ ]:
db_connection.commit()

In [ ]:
chess_db.execute("DROP TABLE xx")

## Chess utilities
Mostly for use in migrating Moves table to GameMoves and PureMoves

In [6]:
def return_db_results(ref_sql):
    """Returns a list of dictionaries from a db table
    where each dictionary is a row of the table, presented
    in the form column_name: value.
    """
    if not ref_sql:
        return None
    prepped_sql = prep_table_request(ref_sql)
    if not prepped_sql:
        return None
    chess_db.execute(prepped_sql)
    PGresults = chess_db.fetchall()
    PGdescription = chess_db.description
    fieldnames = [k[0] for k in PGdescription]
    result_dict = [dict(zip(fieldnames, row)) for row in PGresults]
    return result_dict

def prep_table_request(teststring: str) -> str:
    """Private function to see if user function is requesting 
    just the name of a table, or specific SQL. 
    
    Also vets arbitrary SQL statements to make sure 
    it's a SELECT statement and does not contain a semicolon
    except at the very end. 
    """
    
    # If there are no spaces in the input, it must be just a table
    # name, so just take everything from the table. 
    if teststring.find(' ') < 0:
        teststring = "SELECT * FROM " + teststring
    else:
        # Validate to make sure it's a SELECT statement
        # and that there are no internal semicolons.
        if teststring[:7] != 'SELECT ':
            return ''
        semicolon_pos = teststring.find(';')
        if (semicolon_pos > -1) and semicolon_pos < len(teststring) -1:
            return ''
    return teststring

In [7]:
def get_next_move_ID(tablename: str) -> int:
    """ Returns the next available auto-number key from the given table.
    
        Should never be run with the QuizPositions table, as those rows have 
        their Position (foreign) key as the primary index, not an auto-number key.
    """
    tablefields = {"Moves": "MoveGame", "Games": "GameResult", "Positions": "PositionWhoseMove",
                  "OpeningNodes": "NodePosition", "Quizzes": "QuizDate"}
    sql_string = """INSERT INTO %s (%s) VALUES ('1')""" %(tablename,tablefields[tablename])
    chess_db.execute(sql_string)
    move_id = chess_db.lastrowid
    db_connection.rollback()
    return move_id

def join_FEN_fields(FEN_fields: list)->str:
    fen_string = '/'.join(FEN_fields)
    return fen_string

def make_move_hash(position_1: int, position_2: int) -> str:
    return str(position_1) + "_" + str(position_2)

## FEN Header table 
### Creation and population
These are the standard types of pgn header fields. The original/required/standard ones are given importances of 0 or 1; the fancier/optional/more recent ones have a priority of 2.

In [ ]:
chess_db.execute('''CREATE TABLE FEN_Headers
             (FEN_Headers CHARACTER PRIMARY KEY, FEN_Header_Importance INTEGER)''')

In [ ]:
header_fields = [['Event', 1], ['Site', 1], ['White', 1], ['Black', 1], ['WhiteElo', 2], 
                 ['BlackElo', 2], ['Result', 1], ['TimeControl', 2], ['Date', 1],
                ['EventDate', 0], ['EventType', 0], ['Round', 1], ['ECO', 2], ['EventCountry', 0], 
                 ['EventRounds', 0], ['PlyCount', 2], ['Source', 0], ['SourceDate', 0],
                ['Mode', 0], ['FEN', 0], ['Time', 0], ['Termination', 0], ['Annotator', 2]]

In [ ]:
for header_tag in header_fields:
    chess_db.execute("""INSERT INTO FEN_Headers 
    VALUES ('%s', '%s')"""  %(header_tag[0], header_tag[1]))

In [ ]:
db_connection.commit()

## Game Stubs table
### This is the table of pgn headers for each game. 
At the moment, the program uses these to create the displayed header for each game. It would be nice for all the StubHeaderField values to match the list of FEN_Headers in the table above, but pgn praxis allows them to be arbitrary, and every provider seems to create their own.

In [ ]:
chess_db.execute('''CREATE TABLE Gamestubs
             (StubGameKey INTEGER, StubHeaderField CHARACTER, StubHeaderValue CHARACTER)''')

In [ ]:
db_connection.commit()

## Games table

In [ ]:
chess_db.execute('''CREATE TABLE Games
             (GameKey INTEGER PRIMARY KEY, GameWhitePlayer INTEGER, GameBlackPlayer INTEGER, 
             GameSource INTEGER, GameResult INTEGER, GameSourceType INTEGER, GameRound INTEGER)''')

In [ ]:
# Artificial Player keys for flagging me and repertoires
me_key = 1
white_rep = 5
black_rep = 6

Slav_parent_node = 1
KID_parent_node = 7
english_parent_node = 27

DUMMY_PLAYER = 0 
DUMMY_GAME_SOURCE = 0
GAME_UNDECIDED = -2
GAME_SOURCE_TYPE_REPERTOIRE = 2
NO_GAME_ROUND = 0

In [ ]:
# Creating a game tuple for a repertoire "game" to cover the English opening
game_tuple = (DUMMY_PLAYER, DUMMY_PLAYER, DUMMY_GAME_SOURCE, GAME_UNDECIDED, GAME_SOURCE_TYPE_REPERTOIRE, NO_GAME_ROUND)
chess_db.execute("""INSERT INTO Games (GameWhitePlayer, GameBlackPlayer, GameSource, GameResult, GameSourceType, GameRound)
    VALUES ('%s', '%s', '%s', '%s', '%s', '%s')"""  %(game_tuple))

In [ ]:
db_connection.commit()

## Positions table
### Create table and multi-column index

In [ ]:
chess_db.execute('''CREATE TABLE Positions
             (PositionKey INTEGER PRIMARY KEY, PositionRank8 CHARACTER, PositionRank7 CHARACTER, 
             PositionRank6 CHARACTER, PositionRank5 CHARACTER, PositionRank4 CHARACTER, PositionRank3 CHARACTER, 
             PositionRank2 CHARACTER, PositionRank1 CHARACTER, PositionWhoseMove CHARACTER, 
             PositionCastlingPrivileges CHARACTER)''')

In [ ]:
chess_db.execute('''CREATE INDEX position_idx 
    ON Positions (PositionRank8, PositionRank7, PositionRank6, PositionRank5, PositionRank4, PositionRank3, 
             PositionRank2, PositionRank1, PositionWhoseMove, PositionCastlingPrivileges);''')

### Create first row for the table, the initial position of the game

In [ ]:
initial_position = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'

In [ ]:
def split_fen(hotFEN: str) -> tuple:
    fen_list = hotFEN.split()
    fen_whose_move = fen_list[1]
    fen_castling_privileges = fen_list[2]
    fen_rows = fen_list[0]
    return (fen_rows, fen_whose_move, fen_castling_privileges)

In [ ]:
def get_tuple_for_position_table(positionFEN: str):
    fen_rows_string, fen_whose_move, fen_castling_privileges = split_fen(positionFEN)
    fen_rows_list = fen_rows_string.split("/")
    fen_rows_list.append(fen_whose_move) 
    fen_rows_list.append(fen_castling_privileges) 
    return tuple(fen_rows_list)

In [ ]:
initial_position_fields = get_tuple_for_position_table(initial_position)  
initial_position_fields

In [ ]:
chess_db.execute("""INSERT INTO Positions (PositionRank8, PositionRank7, PositionRank6, PositionRank5, PositionRank4, 
        PositionRank3, PositionRank2, PositionRank1 , PositionWhoseMove, PositionCastlingPrivileges)
    VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')"""  %(initial_position_fields))

In [ ]:
db_connection.commit()

## Moves table

In [ ]:
chess_db.execute('''CREATE TABLE Moves
             (MoveKey INTEGER PRIMARY KEY, MoveGame INTEGER, MoveNumber INTEGER, MoveWhoseMove INTEGER, 
             SquareFrom CHARACTER, SquareTo CHARACTER, MovePromotionPiece CHARACTER, 
             LineLevel CHARACTER, PositionFrom INTEGER, PositionTo INTEGER,
             MoveParent INTEGER, LineParent INTEGER, SublineIndex INTEGER)''')

### Split Moves table into Game Moves and Pure Moves
Game Moves will be the moves with game information like move number and the Game ID. Each contains a relational reference to a Pure Move.
Pure Moves contain the chess information: PositionFrom, PositionTo, notation, etc. Now this can be accessed all at once via a relational reference.

1) Create the new tables

In [3]:
chess_db.execute('''CREATE TABLE PureMoves
             (PureMoveKey INTEGER PRIMARY KEY, WhoseMoveWasThis CHARACTER, 
             SquareFrom CHARACTER, SquareTo CHARACTER, MovePromotionPiece CHARACTER,
             PositionFrom INTEGER, PositionTo INTEGER)''')

In [4]:
chess_db.execute('''CREATE TABLE GameMoves
             (GameMoveKey INTEGER PRIMARY KEY, GameMovePureMove INTEGER, MoveGame INTEGER, 
             MoveNumber INTEGER,  LineParent CHARACTER, SublineIndex CHARACTER)''')

In [5]:
db_connection.commit()

2) Extract the contents of the Moves table into a data structure

In [8]:
move_list = return_db_results("Moves")

In [9]:
# Take a look at the data
move_list[20]

{'MoveKey': 21,
 'MoveGame': 2,
 'MoveNumber': 1,
 'MoveWhoseMove': 'False',
 'SquareFrom': 'g8',
 'SquareTo': 'f6',
 'MovePromotionPiece': '',
 'LineLevel': '0',
 'PositionFrom': 113,
 'PositionTo': 377,
 'MoveParent': 20,
 'LineParent': 0,
 'SublineIndex': 0}

In [10]:
move_record = move_list[20]
make_move_hash(move_record['PositionFrom'], move_record['PositionTo'])

'113_377'

3) Do the actual migration

In [11]:
''' Actual Migration 1
    Populate GameMoves table directly from the old Moves table,
    Build PureMoveDict '''

PureMoveKey = 1
PureMoveDict = {}

for old_move in move_list:
    move_position_hash = make_move_hash(old_move['PositionFrom'], old_move['PositionTo'])
    if move_position_hash not in PureMoveDict:
        PureMoveDict[move_position_hash] = {'PureMoveKey': PureMoveKey, 
            'WhoseMoveWasThis': old_move['MoveWhoseMove'], 'SquareFrom': old_move['SquareFrom'], 
            'SquareTo': old_move['SquareTo'], 'MovePromotionPiece': old_move['MovePromotionPiece'], 
            'PositionFrom': old_move['PositionFrom'], 'PositionTo': old_move['PositionTo']}
        relational_pure_move_key = PureMoveKey
        PureMoveKey += 1
    else:
        relational_pure_move_key = PureMoveDict[move_position_hash]['PureMoveKey']
    move_tuple = (old_move['MoveKey'], relational_pure_move_key, old_move['MoveGame'], 
                  old_move['MoveNumber'],  old_move['LineParent'], old_move['SublineIndex'])
    GameMoveSQL = """INSERT INTO GameMoves (GameMoveKey, GameMovePureMove, MoveGame, 
        MoveNumber, LineParent, SublineIndex)
        VALUES ('%s', '%s', '%s', '%s', '%s', '%s')"""  %(move_tuple)
    chess_db.execute(GameMoveSQL)

In [3]:
db_connection.commit()

In [13]:
# Check data again
PureMoveDict['121_414']

{'PureMoveKey': 61,
 'WhoseMoveWasThis': 'False',
 'SquareFrom': 'c8',
 'SquareTo': 'f5',
 'MovePromotionPiece': '',
 'PositionFrom': 121,
 'PositionTo': 414}

In [14]:
''' Actual Migration 2
    Populate PureMoves table from PureMoveDict '''

for key, value in PureMoveDict.items():
    ''' Besides readability, converting color to w/b letters is better because 
    sqlite stores True/False values as a string.
    '''
    whose_move_was_this = "w" if (value['WhoseMoveWasThis'] == 'True') else "b"
    pure_move_tuple = (value['PureMoveKey'], whose_move_was_this, value['SquareFrom'], 
         value['SquareTo'], value['MovePromotionPiece'], value['PositionFrom'], value['PositionTo'])
    PureMoveSQL = """INSERT INTO PureMoves  (PureMoveKey, WhoseMoveWasThis, SquareFrom, SquareTo, 
        MovePromotionPiece, PositionFrom, PositionTo)
        VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s')"""  %(pure_move_tuple)
    chess_db.execute(PureMoveSQL)

In [2]:
chess_db.execute("DROP TABLE Moves")

## Opening Nodes table

In [ ]:
chess_db.execute('''CREATE TABLE OpeningNodes
             (NodeKey INTEGER PRIMARY KEY, NodeName CHARACTER, NodeAbbrev CHARACTER,  
             NodePositionKey INTEGER, NodeParentNode INTEGER, NodeStemGame INTEGER, 
             NodeIsRoot INTEGER)''')

In [ ]:
db_connection.commit()